<!-- TABS -->
# Fine tune LLM on database

<!-- TABS -->
## Connect to superduper

:::note
Note that this is only relevant if you are running superduper in development mode.
Otherwise refer to "Configuring your production system".
:::

In [2]:
from superduper import superduper

db = superduper('mongomock:///test_db')

2024-Aug-30 21:53:28.08| INFO     | Duncans-MBP.fritz.box| superduper.misc.plugins:13   | Loading plugin: mongodb
2024-Aug-30 21:53:28.21| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:103  | Building Data Layer
2024-Aug-30 21:53:28.21| INFO     | Duncans-MBP.fritz.box| superduper.base.build:171  | Configuration: 
 +---------------+----------------------+
| Configuration |        Value         |
+---------------+----------------------+
|  Data Backend | mongomock:///test_db |
+---------------+----------------------+


<!-- TABS -->
## Get LLM Finetuning Data

The following are examples of training data in different formats.

In [3]:
# <tab: Text>
from datasets import load_dataset
from superduper.base.document import Document
dataset_name = "timdettmers/openassistant-guanaco"
dataset = load_dataset(dataset_name)

train_dataset = dataset["train"]
eval_dataset = dataset["test"]

train_documents = [
    Document({**example, "_fold": "train"})
    for example in train_dataset
]
eval_documents = [
    Document({**example, "_fold": "valid"})
    for example in eval_dataset
]

datas = train_documents + eval_documents

Repo card metadata block was not found. Setting CardData to empty.


In [ ]:
# <tab: Prompt-Response>
from datasets import load_dataset

from superduper.base.document import Document
dataset_name = "mosaicml/instruct-v3"
dataset = load_dataset(dataset_name)

train_dataset = dataset["train"]
eval_dataset = dataset["test"]

train_documents = [
    Document({**example, "_fold": "train"})
    for example in train_dataset
]
eval_documents = [
    Document({**example, "_fold": "valid"})
    for example in eval_dataset
]

datas = train_documents + eval_documents

In [ ]:
# <tab: Chat>
from datasets import load_dataset
from superduper.base.document import Document
dataset_name = "philschmid/dolly-15k-oai-style"
dataset = load_dataset(dataset_name)['train'].train_test_split(0.9)

train_dataset = dataset["train"]
eval_dataset = dataset["test"]

train_documents = [
    Document({**example, "_fold": "train"})
    for example in train_dataset
]
eval_documents = [
    Document({**example, "_fold": "valid"})
    for example in eval_dataset
]

datas = train_documents + eval_documents

We can define different training parameters to handle this type of data.

In [4]:
# <tab: Text>
# Function for transformation after extracting data from the database
transform = None
key = ('text')
training_kwargs=dict(dataset_text_field="text")

In [ ]:
# <tab: Prompt-Response>
# Function for transformation after extracting data from the database
def transform(prompt, response):
    return {'text': prompt + response + "</s>"}

key = ('prompt', 'response')
training_kwargs=dict(dataset_text_field="text")

In [ ]:
# <tab: Chat>
# Function for transformation after extracting data from the database
transform = None

key = ('messages')
training_kwargs=None

Example input_text and output_text

In [5]:
# <tab: Text>
data = datas[0]
input_text, output_text = data["text"].rsplit("### Assistant: ", maxsplit=1)
input_text += "### Assistant: "
output_text = output_text.rsplit("### Human:")[0]
print("Input: --------------")
print(input_text)
print("Response: --------------")
print(output_text)

Input: --------------
### Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.### Assistant: 
Response: --------------
"Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.

Recent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, limited

In [ ]:
# <tab: Prompt-Response>
data = datas[0]
input_text = data["prompt"]
output_text = data["response"]
print("Input: --------------")
print(input_text)
print("Response: --------------")
print(output_text)

In [ ]:
# <tab: Chat>
data = datas[0]
messages = data["messages"]
input_text = messages[:-1]
output_text = messages[-1]["content"]
print("Input: --------------")
print(input_text)
print("Response: --------------")
print(output_text)

<!-- TABS -->
## Insert simple data

After turning on auto_schema, we can directly insert data, and superduper will automatically analyze the data type, and match the construction of the table and datatype.

In [6]:
from superduper import Document

table_or_collection = db['docs']

ids = db.execute(table_or_collection.insert([Document(data) for data in datas]))
select = table_or_collection.select()

2024-Aug-30 21:53:39.13| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:363  | Table docs does not exist, auto creating...
2024-Aug-30 21:53:39.13| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:369  | Creating table docs with schema {('_fold', 'str'), ('text', 'str')}
2024-Aug-30 21:53:39.13| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:415  | Leaf str already exists
2024-Aug-30 21:53:40.86| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:344  | Inserted 10364 documents into docs


## Select a Model

In [7]:
model_name = "facebook/opt-125m"
model_kwargs = dict()
tokenizer_kwargs = dict()

# or 
# model_name = "mistralai/Mistral-7B-Instruct-v0.2"
# token = "hf_xxxx"
# model_kwargs = dict(token=token)
# tokenizer_kwargs = dict(token=token)

<!-- TABS -->
## Build A Trainable LLM

**Create an LLM Trainer for training**

The parameters of this LLM Trainer are basically the same as `transformers.TrainingArguments`, but some additional parameters have been added for easier training setup.

In [8]:
from superduper_transformers import LLM, LLMTrainer

trainer = LLMTrainer(
    identifier="llm-finetune-trainer",
    output_dir="output/finetune",
    overwrite_output_dir=True,
    num_train_epochs=3,
    save_total_limit=3,
    logging_steps=10,
    evaluation_strategy="steps",
    save_steps=100,
    eval_steps=100,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    max_seq_length=512,
    key=key,
    select=select,
    transform=transform,
    training_kwargs=training_kwargs,
)

In [ ]:
# <tab: Lora>
trainer.use_lora = True

In [ ]:
# <tab: QLora>
trainer.use_lora = True
trainer.bits = 4

In [ ]:
# <tab: Deepspeed>
!pip install deepspeed
deepspeed = {
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": "auto",
    "gradient_accumulation_steps": "auto",
    "zero_optimization": {
        "stage": 2,
    },
}
trainer.use_lora = True
trainer.bits = 4
trainer.deepspeed = deepspeed

In [ ]:
# <tab: Multi-GPUS>
trainer.use_lora = True
trainer.bits = 4
trainer.num_gpus = 2

Create a trainable LLM model and add it to the database, then the training task will run automatically.

In [9]:
llm = LLM(
    identifier="llm",
    model_name_or_path=model_name,
    trainer=trainer,
    model_kwargs=model_kwargs,
    tokenizer_kwargs=tokenizer_kwargs,
)

db.apply(llm)

2024-Aug-30 21:53:51.59| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:415  | Leaf dill already exists
2024-Aug-30 21:53:51.59| WARNING  | Duncans-MBP.fritz.box| superduper.backends.local.artifacts:85   | File /tmp/test_db/4a8dc14137b3a79a81256a795b266fe82bda52d9 already exists


(['36869320-20a5-4cb3-ac4d-d40c146146ca'],
 LLM(trainer=LLMTrainer(identifier='llm-finetune-trainer', uuid='d6c6390a57a4446797bb5250549c7077', upstream=None, plugins=None, cache=False, key='text', select=docs.select(), transform=None, metric_values={}, signature='*args', data_prefetch=False, prefetch_size=1000, prefetch_factor=100, in_memory=True, compute_kwargs={}, output_dir='output/finetune', overwrite_output_dir=True, do_train=False, do_eval=False, do_predict=False, eval_strategy='no', prediction_loss_only=False, per_device_train_batch_size=1, per_device_eval_batch_size=1, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=2, eval_accumulation_steps=None, eval_delay=0, torch_empty_cache_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3, max_steps=-1, lr_scheduler_type='linear', lr_scheduler_kwargs={}, warmup_ratio=0.0, warmup_steps=0, log_level='passive

## Load the trained model
There are two methods to load a trained model:

- **Load the model directly**: This will load the model with the best metrics (if the transformers' best model save strategy is set) or the last version of the model.
- **Use a specified checkpoint**: This method downloads the specified checkpoint, then initializes the base model, and finally merges the checkpoint with the base model. This approach supports custom operations such as resetting flash_attentions, model quantization, etc., during initialization.

In [10]:
# <tab: Load Trained Model Directly>
llm = db.load("model", "llm")

In [ ]:
# <tab: Use a specified checkpoint>
from superduper_transformers import LLM

experiment_id = db.show("checkpoint")[-1]
version = None # None means the last checkpoint
checkpoint = db.load("checkpoint", experiment_id, version=version)
llm = LLM(
    identifier="llm",
    model_name_or_path=model_name,
    adapter_id=checkpoint,
    model_kwargs=dict(load_in_4bit=True)
)

In [11]:
llm.predict(input_text, max_new_tokens=200)

/Users/dodo/.pyenv/versions/3.11.7/envs/superduper-3.11/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


'                                                                                                                                                                                                        '

In [12]:
from superduper import Template

t = Template('llm-finetune', template=llm, substitutions={'docs': 'collection', model_name: 'model_name'})

2024-Aug-30 21:54:29.36| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:415  | Leaf dill already exists


In [13]:
t.export('.')